# U.S. Small Business Administration High Risk Loan Applicant Analysis

## Data Wrangling & Cleaning - Part 2

### Loading Libraries and Datasets

In [1]:
import numpy as np
import pandas as pd
import us

from library.utils import save_file

To deepen this analysis, outside economic factors should be considered. Small businesses are likely most affected by their community economy, the greatest. Therefore, economic data around the counties could be very impactful. While county data is not currently in the data set, that information can be obtained with the zip code. Unfortunately, annual data by county prior to 2010 is very incomplete. Most data was in five year estimates and the data usually only populated for counties when populations were greater than 65,000. Therefore, some national level and state level data will be utilized instead. Information such as the gross domestic product, inflation rate, and unemployment data can be useful indicators for when the economy is struggling and small businesses are usually the first to struggle. Those three data points are going to be added to the dataset that was completed in the previous data wrangling notebook.

In [2]:
gdp_df = pd.read_csv('../data/raw/united-states-gdp-growth-rate.csv')
inf_rate_df = pd.read_csv('../data/raw/united-states-inflation-rate-cpi.csv')
state_unemp_data = pd.read_excel(io='../data/raw/emp-unemployment.xls', sheet_name='States')
apps_data = pd.read_csv('../data/interim/sba_national_summary.csv')

### Gross Domestic Product

This dataset provides the percent growth and annual change dating all the way back to 1961.

In [3]:
gdp_df.head()

,date,GDP Growth (%),Annual Change,
0,1961-12-31,2.3,NaN,NaN
1,1962-12-31,6.1,3.8,NaN
2,1963-12-31,4.4,-1.7,NaN
3,1964-12-31,5.8,1.4,NaN
4,1965-12-31,6.4,0.6,NaN


**GDP Cleanup**

Focusing on renaming columns to simpler labels, dropping the unused column at the end, removing na values, and reformatting the date to be just the year.

In [4]:
gdp_df.columns

Index(['date', ' GDP Growth (%)', ' Annual Change', ' '], dtype='object')

In [5]:
gdp_df = gdp_df.drop(columns=[' '], axis=1)

In [6]:
gdp_df = gdp_df.rename(columns={'date': 'year', ' GDP Growth (%)': 'gdp_growth', ' Annual Change': 'gdp_annual_change'})

In [7]:
gdp_df['year'] = pd.to_datetime(gdp_df['year']).dt.year

In [8]:
gdp_df['gdp_annual_change'] = gdp_df['gdp_annual_change'].fillna(0)

In [9]:
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               60 non-null     int64  
 1   gdp_growth         60 non-null     float64
 2   gdp_annual_change  60 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.5 KB


In [10]:
gdp_df.head()

,year,gdp_growth,gdp_annual_change
0,1961,2.3,0.0
1,1962,6.1,3.8
2,1963,4.4,-1.7
3,1964,5.8,1.4
4,1965,6.4,0.6


### Inflation Rate

This dataset provides information on the inflation rate and its annual change all the way back to 1960.

In [11]:
inf_rate_df.head()

,date,Inflation Rate (%),Annual Change,
0,1960-12-31,1.4580,NaN,NaN
1,1961-12-31,1.0707,-0.39,NaN
2,1962-12-31,1.1988,0.13,NaN
3,1963-12-31,1.2397,0.04,NaN
4,1964-12-31,1.2789,0.04,NaN


**Inflation Rate Cleanup**

Focusing on renaming columns to simpler labels, dropping the unused column at the end, removing na values, and reformatting the date to be just the year.

In [12]:
inf_rate_df.columns

Index(['date', ' Inflation Rate (%)', ' Annual Change', ' '], dtype='object')

In [13]:
inf_rate_df = inf_rate_df.drop(columns=[' '], axis=1)
inf_rate_df = inf_rate_df.rename(columns={'date': 'year', ' Inflation Rate (%)': 'inflation_rate', ' Annual Change': 'inf_rate_annual_chg'})
inf_rate_df.head()

,year,inflation_rate,inf_rate_annual_chg
0,1960-12-31,1.4580,NaN
1,1961-12-31,1.0707,-0.39
2,1962-12-31,1.1988,0.13
3,1963-12-31,1.2397,0.04
4,1964-12-31,1.2789,0.04


In [14]:
inf_rate_df['year'] = pd.to_datetime(inf_rate_df['year']).dt.year
inf_rate_df['inf_rate_annual_chg'] = inf_rate_df['inf_rate_annual_chg'].fillna(0)

In [15]:
inf_rate_df.head()

,year,inflation_rate,inf_rate_annual_chg
0,1960,1.4580,0.00
1,1961,1.0707,-0.39
2,1962,1.1988,0.13
3,1963,1.2397,0.04
4,1964,1.2789,0.04


In [16]:
inf_rate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 61 non-null     int64  
 1   inflation_rate       61 non-null     float64
 2   inf_rate_annual_chg  61 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.6 KB


### State Unemployment Data

This dataset provides the annual unemployment rate by state dating back to 1980. State unemployment data includes all states except for Wyoming. Therefore, Wyoming will not be weighed with this data

In [17]:
state_unemp_data.columns = state_unemp_data.iloc[4]
state_unemp_data = state_unemp_data.iloc[6:56, :]
state_unemp_data

4,Fips,Area,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,...,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
6,01000,Alabama,8.9,10.6,14.1,13.8,11.0,9.2,9.7,8.1,...,11.0,10.5,9.6,8.0,7.2,6.8,6.1,5.8,4.4,3.9
7,02000,Alaska,9.6,9.4,9.9,9.9,9.8,9.7,10.9,10.3,...,7.7,7.9,7.6,7.1,7.0,6.9,6.5,6.9,7.0,6.6
8,04000,Arizona,6.6,6.2,10.1,8.8,5.2,6.3,6.9,6.5,...,9.9,10.4,9.5,8.3,7.7,6.8,6.1,5.4,4.9,4.8
9,05000,Arkansas,7.6,8.7,9.9,9.9,8.7,8.7,8.6,8.1,...,7.8,8.2,8.3,7.6,7.2,6.0,5.0,4.0,3.7,3.7
10,06000,California,6.8,7.4,10.0,9.8,7.8,7.2,6.7,5.8,...,11.2,12.2,11.7,10.4,8.9,7.5,6.2,5.5,4.8,4.2
11,08000,Colorado,5.8,5.7,7.3,7.1,5.4,6.1,7.5,7.5,...,7.3,8.7,8.4,7.9,6.9,5.0,3.9,3.2,2.7,3.3
12,09000,Connecticut,5.8,6.2,6.9,6.0,4.7,4.7,3.8,3.4,...,7.9,9.1,8.8,8.3,7.8,6.6,5.7,5.1,4.7,4.1
13,10000,Delaware,7.6,7.8,8.4,7.7,6.3,5.1,4.3,3.1,...,8.3,8.4,7.5,7.2,6.7,5.7,4.9,4.5,4.5,3.8
14,11000,District of Columbia,7.4,8.9,10.4,11.2,9.1,8.1,7.5,6.3,...,9.3,9.4,10.2,9.0,8.5,7.8,6.9,6.1,6.1,5.6
15,12000,Florida,6.1,6.7,8.4,8.5,6.4,6.0,5.8,5.3,...,10.4,11.1,10.0,8.5,7.2,6.3,5.5,4.8,4.2,3.6


**Unemployment Data Cleanup**

The *Fips* column will not be needed. In the main loan applicant dataset, the state value is the two letter acronym. The **us** library will be used to translate the *Area* state name to a new *state* two letter acronym. The *Area* column will then be dropped.

After that, the year is very useful information. This dataset will be melted down to where the year will be a column and all unemployment rate data will be its own column.

Also, this dataset is missing one state: Wyoming. To fill in this data, the mean of all the states will be used.

In [18]:
state_unemp_data['state'] = state_unemp_data['Area'].apply(lambda x: us.states.lookup(x).abbr)
state_unemp_data = state_unemp_data.drop(columns=['Fips', 'Area'], axis=1)

In [19]:
wyoming_data = pd.Series(state_unemp_data.mean(axis=0))
wyoming_data = pd.concat([wyoming_data, pd.Series({'state': 'WY'})]).to_frame().T
wyoming_data

/var/folders/lh/2pt476ks5r5_ln7lg89jxxbh0000gn/T/ipykernel_37801/4144235439.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  wyoming_data = pd.Series(state_unemp_data.mean(axis=0))


,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,state
0,6.876,7.378,9.28,9.176,7.324,7.066,6.904,6.156,5.432,5.11,...,8.808,8.21,7.41,6.806,5.816,5.064,4.662,4.188,3.806,WY


In [20]:
state_unemp_data = pd.concat([state_unemp_data, wyoming_data])

In [21]:
state_unemp_data

,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,state
6,8.9,10.6,14.1,13.8,11.0,9.2,9.7,8.1,7.2,7.0,...,10.5,9.6,8.0,7.2,6.8,6.1,5.8,4.4,3.9,AL
7,9.6,9.4,9.9,9.9,9.8,9.7,10.9,10.3,8.8,7.0,...,7.9,7.6,7.1,7.0,6.9,6.5,6.9,7.0,6.6,AK
8,6.6,6.2,10.1,8.8,5.2,6.3,6.9,6.5,6.4,5.3,...,10.4,9.5,8.3,7.7,6.8,6.1,5.4,4.9,4.8,AZ
9,7.6,8.7,9.9,9.9,8.7,8.7,8.6,8.1,7.6,7.0,...,8.2,8.3,7.6,7.2,6.0,5.0,4.0,3.7,3.7,AR
10,6.8,7.4,10.0,9.8,7.8,7.2,6.7,5.8,5.3,5.1,...,12.2,11.7,10.4,8.9,7.5,6.2,5.5,4.8,4.2,CA
11,5.8,5.7,7.3,7.1,5.4,6.1,7.5,7.5,6.5,5.7,...,8.7,8.4,7.9,6.9,5.0,3.9,3.2,2.7,3.3,CO
12,5.8,6.2,6.9,6.0,4.7,4.7,3.8,3.4,3.0,3.8,...,9.1,8.8,8.3,7.8,6.6,5.7,5.1,4.7,4.1,CT
13,7.6,7.8,8.4,7.7,6.3,5.1,4.3,3.1,3.1,3.5,...,8.4,7.5,7.2,6.7,5.7,4.9,4.5,4.5,3.8,DE
14,7.4,8.9,10.4,11.2,9.1,8.1,7.5,6.3,5.0,4.9,...,9.4,10.2,9.0,8.5,7.8,6.9,6.1,6.1,5.6,DC
15,6.1,6.7,8.4,8.5,6.4,6.0,5.8,5.3,5.1,5.6,...,11.1,10.0,8.5,7.2,6.3,5.5,4.8,4.2,3.6,FL


In [22]:
state_unemp_data_melt = pd.melt(state_unemp_data, id_vars=['state'], var_name='year', value_name='unemployment_rate')

In [23]:
state_unemp_data_melt['year'] = state_unemp_data_melt['year'].astype(int)

In [24]:
state_unemp_data_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   state              1989 non-null   object
 1   year               1989 non-null   int64 
 2   unemployment_rate  1989 non-null   object
dtypes: int64(1), object(2)
memory usage: 46.7+ KB


In [25]:
state_unemp_data_melt.head()

,state,year,unemployment_rate
0,AL,1980,8.9
1,AK,1980,9.6
2,AZ,1980,6.6
3,AR,1980,7.6
4,CA,1980,6.8


### Merging Data

Merging the State Unemployment Data to the application data.

In [26]:
apps_data_ver2 = apps_data.merge(state_unemp_data_melt, how='left', 
                                 left_on=['State', 'ApprovalFY'], 
                                 right_on=['state', 'year'],
                                )

In [27]:
apps_data_ver2

,City,State,Zip,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,...,GrAppv,SBA_Appv,NAICS_sectors,isFranchise,RevLineCr_v2,LowDoc_v2,MIS_Status_v2,state,year,unemployment_rate
0,EVANSVILLE,IN,47711,1997,84,4,new_business,0,0,unknown,...,60000.0,48000.0,45,not_franchise,N,Y,paid,IN,1997.0,3.5
1,NEW PARIS,IN,46526,1997,60,2,new_business,0,0,unknown,...,40000.0,32000.0,72,not_franchise,N,Y,paid,IN,1997.0,3.5
2,BLOOMINGTON,IN,47401,1997,180,7,existing_business,0,0,unknown,...,287000.0,215250.0,62,not_franchise,N,N,paid,IN,1997.0,3.5
3,BROKEN ARROW,OK,74012,1997,60,2,existing_business,0,0,unknown,...,35000.0,28000.0,0,not_franchise,N,Y,paid,OK,1997.0,4.1
4,ORLANDO,FL,32801,1997,240,14,existing_business,7,7,unknown,...,229000.0,229000.0,0,not_franchise,N,N,paid,FL,1997.0,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897159,UPPER ARLINGTON,OH,43221,1997,60,6,existing_business,0,0,unknown,...,70000.0,56000.0,45,not_franchise,0,N,paid,OH,1997.0,4.6
897160,COLUMBUS,OH,43221,1997,60,6,existing_business,0,0,unknown,...,85000.0,42500.0,45,not_franchise,Y,N,paid,OH,1997.0,4.6
897161,SANTA MARIA,CA,93455,1997,108,26,existing_business,0,0,unknown,...,300000.0,225000.0,33,not_franchise,N,N,paid,CA,1997.0,6.4
897162,HONOLULU,HI,96830,1997,60,6,existing_business,0,0,unknown,...,75000.0,60000.0,0,not_franchise,N,Y,default,HI,1997.0,6.2


Unemployment, year, and state was successfully merged to the apps_data dataframe.

Next, the GDP and Inflation Rate datasets are very similar. Those two datasets can be joined together and then their merged dataset will be added to the loan apps dataset. 

In [28]:
gdp_inf_combined = gdp_df.merge(inf_rate_df, how='inner', on='year')

In [29]:
gdp_inf_combined.head()

,year,gdp_growth,gdp_annual_change,inflation_rate,inf_rate_annual_chg
0,1961,2.3,0.0,1.0707,-0.39
1,1962,6.1,3.8,1.1988,0.13
2,1963,4.4,-1.7,1.2397,0.04
3,1964,5.8,1.4,1.2789,0.04
4,1965,6.4,0.6,1.5852,0.31


In [30]:
apps_data_ver3 = apps_data_ver2.merge(gdp_inf_combined, how='left', 
                                 left_on='ApprovalFY', 
                                 right_on='year',
                                )

In [31]:
apps_data_ver3.head()

,City,State,Zip,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,...,LowDoc_v2,MIS_Status_v2,state,year_x,unemployment_rate,year_y,gdp_growth,gdp_annual_change,inflation_rate,inf_rate_annual_chg
0,EVANSVILLE,IN,47711,1997,84,4,new_business,0,0,unknown,...,Y,paid,IN,1997.0,3.5,1997,4.4472,0.67,2.3377,-0.59
1,NEW PARIS,IN,46526,1997,60,2,new_business,0,0,unknown,...,Y,paid,IN,1997.0,3.5,1997,4.4472,0.67,2.3377,-0.59
2,BLOOMINGTON,IN,47401,1997,180,7,existing_business,0,0,unknown,...,N,paid,IN,1997.0,3.5,1997,4.4472,0.67,2.3377,-0.59
3,BROKEN ARROW,OK,74012,1997,60,2,existing_business,0,0,unknown,...,Y,paid,OK,1997.0,4.1,1997,4.4472,0.67,2.3377,-0.59
4,ORLANDO,FL,32801,1997,240,14,existing_business,7,7,unknown,...,N,paid,FL,1997.0,4.8,1997,4.4472,0.67,2.3377,-0.59


Now that all the data is merged, it is time to remove the duplicated columns. There is now two state columns and three year columns. The columns added by the outside datasets (state, year_x, year_y) will be dropped.

In [32]:
apps_data_ver4 = apps_data_ver3.drop(columns=['state','year_x', 'year_y'], axis=1)
apps_data_ver4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897164 entries, 0 to 897163
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   City                 897164 non-null  object 
 1   State                897164 non-null  object 
 2   Zip                  897164 non-null  int64  
 3   ApprovalFY           897164 non-null  int64  
 4   Term                 897164 non-null  int64  
 5   NoEmp                897164 non-null  int64  
 6   NewExist             897164 non-null  object 
 7   CreateJob            897164 non-null  int64  
 8   RetainedJob          897164 non-null  int64  
 9   UrbanRural           897164 non-null  object 
 10  DisbursementGross    897164 non-null  float64
 11  BalanceGross         897164 non-null  float64
 12  ChgOffPrinGr         897164 non-null  float64
 13  GrAppv               897164 non-null  float64
 14  SBA_Appv             897164 non-null  float64
 15  NAICS_sectors    

The Unmployment_rate column in missing data. This is likely due to the application occurring before 1980.

In [33]:
apps_data_ver4['unemployment_rate'].isnull().sum()

984

In [34]:
(apps_data_ver4['ApprovalFY'] < 1980).sum()

983

There is only one row unaccounted for here. Data occurring before 1980 will not have a much of an impact for indicators. Therefore those rows will be dropped. However, for the other row...

In [35]:
apps_data_ver4[(apps_data_ver4['unemployment_rate'].isnull()) & (apps_data_ver4['ApprovalFY'] >= 1980)]

,City,State,Zip,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,...,NAICS_sectors,isFranchise,RevLineCr_v2,LowDoc_v2,MIS_Status_v2,unemployment_rate,gdp_growth,gdp_annual_change,inflation_rate,inf_rate_annual_chg
305117,"BOX 267, APO AP",AP,96205,1988,73,8,existing_business,0,0,unknown,...,54,not_franchise,N,N,default,NaN,4.177,0.72,4.0777,0.41


This row is very interesting. The application occurs with Armed Forces Pacific which means it is a military zip code designated for the representations in other countries. This one specifically represents Slovakia. Considering there is only one instance of this (although, this loan did default), this row will be dropped. There are not enough observations with instances like these to be reliable.

In [36]:
apps_data_ver5 = apps_data_ver4[apps_data_ver4['unemployment_rate'].notnull()]

In [37]:
apps_data_ver5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896180 entries, 0 to 897163
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   City                 896180 non-null  object 
 1   State                896180 non-null  object 
 2   Zip                  896180 non-null  int64  
 3   ApprovalFY           896180 non-null  int64  
 4   Term                 896180 non-null  int64  
 5   NoEmp                896180 non-null  int64  
 6   NewExist             896180 non-null  object 
 7   CreateJob            896180 non-null  int64  
 8   RetainedJob          896180 non-null  int64  
 9   UrbanRural           896180 non-null  object 
 10  DisbursementGross    896180 non-null  float64
 11  BalanceGross         896180 non-null  float64
 12  ChgOffPrinGr         896180 non-null  float64
 13  GrAppv               896180 non-null  float64
 14  SBA_Appv             896180 non-null  float64
 15  NAICS_sectors    

The loans app dataset has been properly integrated with 5 new key pieces of information that can provide some very useful insights as to which loan applicants would be at a high risk of defaulting. The next step is start looking at these individual features to determine the relationships that they have with the target feature, **MIS_Status_v2**.

In [38]:
datapath = '../data/interim'
save_file(apps_data_ver5, 'sba_national_final_ver1.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../data/interim/sba_national_final_ver1.csv"
